In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2 as cv
import imutils
import easyocr

import os

# Help Functions

In [3]:
# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i - 1] for i in net.getUnconnectedOutLayers()]

In [4]:
# Draw the predicted bounding box
def drawPred(classId, conf, left, top, right, bottom):
    # Draw a bounding box.
    #    cv.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    cv.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 3)

    label = '%.2f' % conf

    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s: %s' % (classes[classId], label)

    # Display the label at the top of the bounding box
    labelSize, baseLine = cv.getTextSize(
        label, cv.FONT_HERSHEY_SIMPLEX, 1, 1)
    top = max(top, labelSize[1])
    #cv.rectangle(frame, (left, top - round(1.7*labelSize[1])), (left + round(
    #   1.3*labelSize[0]), top + baseLine), (255, 0, 255), cv.FILLED)
    # cv.putText(frame, label, (left, top),
    #           cv.FONT_HERSHEY_SIMPLEX, 1.3, (255, 255, 255), 2)

In [5]:
# Remove the bounding boxes with low confidence using non-maxima suppression
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        print("out.shape : ", out.shape)
        for detection in out:
            # if detection[4]>0.001:
            scores = detection[5:]
            classId = np.argmax(scores)
            # if scores[classId]>confThreshold:
            confidence = scores[classId]
            if detection[4] > confThreshold:
                print(detection[4], " - ", scores[classId],
                      " - th : ", confThreshold)
                print(f"detection: {detection}")
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    return indices, boxes, classIds, confidences

In [42]:
def opencv_license_plates_localization(file_path, reader, output_file_path=None):
    
    # Extract the filename
    file_name = os.path.basename(file_path)
    
    img = cv.imread(file_path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY) # Make the image gray
    
    bfilter = cv.bilateralFilter(gray, 11, 17, 17) # Noise reduction
    edged = cv.Canny(bfilter, 30, 200) # Edge detection
    
    keypoints = cv.findContours(edged.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE) # Find keypoints
    contours = imutils.grab_contours(keypoints) # Grab contours
    contours = sorted(contours, key=cv.contourArea, reverse=True)[:30] # Select the 10 largest contours 
    
    # Find the position of the license plate (rectangle search)
    location = None
    for contour in contours:
        approx = cv.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break
    
    if location is None:
        return None
        
    # Extract the license plate from the original image
    mask = np.zeros(gray.shape, np.uint8) # create a blank mask
    new_image = cv.drawContours(mask, [location], 0, 255, -1) # draw contours inside the mask image with location coordinates
    new_image = cv.bitwise_and(img, img, mask=mask) # Applying masks on top of the original image
    (x, y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2 + 1, y1:y2 + 1]
    result = reader.readtext(cropped_image)
    if result != []:
        text = result[0][-2]
        (x, y, w, h) = cv.boundingRect(location)
        # Create a sign image using (x, y, w, h) information
        sign_image = img[y:y+h, x:x+w]
        
        temp_image = img.copy()
        res_image = cv.rectangle(temp_image, tuple(approx[0][0]), tuple(approx[2][0]), (0, 255, 0), 3) # draw green rectangle
        
        # Create a figure with two subplots: one for the car image and one for the license plate image
        fig, axes = plt.subplots(1, 3, figsize=(12, 6))
        
        # Display the car image on the left subplot
        axes[0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        axes[0].set_title(file_name)

        # Display the license plate image in the middle subplot
        axes[1].imshow(cv.cvtColor(cropped_image, cv.COLOR_BGR2RGB))
        axes[1].set_title(file_name)

        # Display the sign image on the right subplot
        axes[2].imshow(cv.cvtColor(res_image, cv.COLOR_BGR2RGB))
        axes[2].set_title(f"OpenCV: {text}")

        plt.show()  # Display the current image
            
        if text != '':
            if output_file_path:
                cv.imwrite(output_file_path, sign_image)
            return (x, y, w, h)

# Initialize Yolo model

In [6]:
# Initialize the parameters
confThreshold = 0.5  # Confidence threshold
nmsThreshold = 0.4  # Non-maximum suppression threshold

inpWidth = 416  # 608     # Width of network's input image
inpHeight = 416  # 608     # Height of network's input image

In [7]:
# Load names of classes
classesFile = "yolo-license-plate-detection/model/classes.names"

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "yolo-license-plate-detection/model/config/darknet-yolov3.cfg"
modelWeights = "yolo-license-plate-detection/model/weights/model.weights"

net = cv.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# Use Yolo model

In [ ]:
image_dir = 'data/images/many_cars'

output_dir = 'data/plates/many_cars/'

if not os.path.isdir(image_dir):
    print("Input image dir ", image_dir, " doesn't exist")
    
reader = easyocr.Reader(['en'])

license_plates_dict = {'image_name': [], 'x': [], 'y': [], 'width': [], 'height': [] }
for image_name in [k for k in os.listdir(image_dir) if 'out_py' not in k]:
    image_path = os.path.join(image_dir, image_name)
    print(f"Image_path = {image_path}")
    
    output_img_path = os.path.join(output_dir, image_name)
    
    image = cv.imread(image_path)
    frame = image.copy()
    # Create a 4D blob from the frame
    blob = cv.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0, 0, 0], 1, crop=False)
    
    # Set input to the network
    net.setInput(blob)

    # Run forward pass and get output
    outs = net.forward(getOutputsNames(net))

    # Remove bounding boxes with low confidence and draw predictions
    indices, boxes, classIds, confidences = postprocess(frame, outs)
    if len(indices) != 0:
        # Loop through detected boxes and find the license plate
        for i in indices:
            box = boxes[i]
            left, top, width, height = box[0], box[1], box[2], box[3]
            classId = classIds[i]

            # Check if the detected object is a license plate
            if classes[classId] == 'License Plate':
                print(f"left, top, width, height = {left, top, width, height}")
                # Extract the license plate region
                if left < 0:
                    left = 0
                license_plate = frame[top:top+height, left:left+width]
                result = reader.readtext(license_plate)
                if result != []:
                    text = result[0][-2]
                    
                    if text != '':
                        license_plates_dict['image_name'].append(image_name)
                        license_plates_dict['x'].append(left)
                        license_plates_dict['y'].append(top)
                        license_plates_dict['width'].append(width)
                        license_plates_dict['height'].append(height)
                        cv.imwrite(output_img_path, license_plate)
                        
                    # Display the original image, license plate, and license plate on the original image
                    fig, axes = plt.subplots(1, 3, figsize=(12, 6))
                    axes[0].imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
                    axes[0].set_title('Original Image')
        
                    axes[1].imshow(cv.cvtColor(license_plate, cv.COLOR_BGR2RGB))
                    axes[1].set_title('License Plate')
                
                    drawPred(classIds[i], confidences[i], left, top, left + width, top + height)
                    axes[2].imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
                    axes[2].set_title(f"YOLO: {text}")
        
                    plt.show()
            
                else:
                    print(f"\nNO TEXT WAS DETECTED ON THE SELECTED PART OF THE IMAGE: {image_name}!!!\n")

    else:
        result = opencv_license_plates_localization(file_path=image_path, reader=reader, output_file_path=output_img_path)
        if result is None:
            print(f"\nNO LICENSE PLATE DETECTED IN THIS IMAGE: {image_name}!!!\n")
            plt.imshow(cv.cvtColor(image, cv.COLOR_BGR2RGB))
            plt.title(image_name)
            plt.show()
        else:
            left, top, width, height = result[0], result[1], result[2], result[3]
            license_plates_dict['image_name'].append(image_name)
            license_plates_dict['x'].append(left)
            license_plates_dict['y'].append(top)
            license_plates_dict['width'].append(width)
            license_plates_dict['height'].append(height)

## Create a licenses_df

In [79]:
current_iteration_licenses_df = pd.DataFrame.from_dict(license_plates_dict)
current_iteration_licenses_df.head()

,image_name,x,y,width,height
0,Cars104.png,194,266,46,14
1,Cars211.png,0,269,177,22
2,Cars215.png,100,105,62,16
3,Cars223.png,342,227,68,21
4,Cars223.png,482,268,69,32


In [80]:
current_iteration_licenses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  14 non-null     object
 1   x           14 non-null     int64 
 2   y           14 non-null     int64 
 3   width       14 non-null     int64 
 4   height      14 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 688.0+ bytes


In [81]:
licenses_df = pd.concat([licenses_df, current_iteration_licenses_df])
licenses_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 13
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  233 non-null    object
 1   x           233 non-null    int64 
 2   y           233 non-null    int64 
 3   width       233 non-null    int64 
 4   height      233 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 10.9+ KB


In [82]:
licenses_df.to_csv('data/dataframes/cars.csv', index=False)

In [83]:
test_licenses_df = pd.read_csv('data/dataframes/cars.csv')
test_licenses_df.head()

,image_name,x,y,width,height
0,Cars0.png,256,127,153,46
1,Cars1.png,144,126,113,27
2,Cars11.png,130,199,142,38
3,Cars14.png,98,117,187,39
4,Cars18.png,312,281,94,35


In [84]:
test_licenses_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  233 non-null    object
 1   x           233 non-null    int64 
 2   y           233 non-null    int64 
 3   width       233 non-null    int64 
 4   height      233 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 9.2+ KB
